### 0 loading libs & competition data

In [20]:
import pandas as pd    
%run cfcs.py

In [21]:
riskfutures = pd.read_csv('Data/corn_climate_risk_futures_daily_master.csv')
marketshare = pd.read_csv('Data/corn_regional_market_share.csv')
## dir need to change before submission to Kaggle input directories

#### Note: EDAs were done seperately.

### 1 Baseline Feature Engineering

In [22]:
mergedf = riskfutures.copy()
mergedf['day_of_year'] = pd.to_datetime(mergedf['date_on'],format='%Y-%m-%d').dt.dayofyear
mergedf['quarter'] = pd.to_datetime(mergedf['date_on'],format='%Y-%m-%d').dt.quarter

In [23]:
mergedf = mergedf.merge(marketshare[['region_id','percent_country_production']],how='left',on='region_id')

In [24]:
mergedf['percent_country_production'] = mergedf['percent_country_production'].fillna(0.0)
## see EDA_regional_marketshare.ipynb

#### New Climate Features Risks

In [25]:
# Total cnt locations for each rows
mergedf['total_location_by_region'] = mergedf['climate_risk_cnt_locations_heat_stress_risk_low'] + \
                                    mergedf['climate_risk_cnt_locations_heat_stress_risk_medium'] + \
                                    mergedf['climate_risk_cnt_locations_heat_stress_risk_high']

# Climate Risk for Coldwave, and Flood:
for i in range(1, 5):
    mergedf[f'medium_coldstress_lag_{i}'] = mergedf['climate_risk_cnt_locations_unseasonably_cold_risk_medium'].shift(i)
    mergedf[f'medium_precip_lag_{i}'] = mergedf['climate_risk_cnt_locations_excess_precip_risk_medium'].shift(i)
    
    mergedf[f'medium_coldstress_lag_{i}'] = mergedf[f'medium_coldstress_lag_{i}'].fillna(0)
    mergedf[f'medium_precip_lag_{i}'] = mergedf[f'medium_precip_lag_{i}'].fillna(0)

for j in range(1, 3): 
    mergedf[f'high_coldstress_lag_{j}'] = mergedf['climate_risk_cnt_locations_unseasonably_cold_risk_high'].shift(j)
    mergedf[f'high_precip_lag_{j}'] = mergedf['climate_risk_cnt_locations_excess_precip_risk_high'].shift(j)

    mergedf[f'high_coldstress_lag_{j}'] = mergedf[f'high_coldstress_lag_{j}'].fillna(0)
    mergedf[f'high_precip_lag_{j}'] = mergedf[f'high_precip_lag_{j}'].fillna(0)

mergedf['medium_coldstress_4days_average'] = mergedf[[f'medium_coldstress_lag_{i}' for i in range(1, 5)]].mean(axis=1)
mergedf['medium_coldstress_2days_average'] = mergedf[[f'medium_coldstress_lag_{i}' for i in range(1, 3)]].mean(axis=1)
mergedf['high_coldstress_2days_average'] = mergedf[[f'high_coldstress_lag_{i}' for i in range(1, 3)]].mean(axis=1)

mergedf['medium_precip_4days_average'] = mergedf[[f'medium_precip_lag_{i}' for i in range(1, 5)]].mean(axis=1)
mergedf['medium_precip_2days_average'] = mergedf[[f'medium_precip_lag_{i}' for i in range(1, 3)]].mean(axis=1)
mergedf['high_precip_2days_average'] = mergedf[[f'high_precip_lag_{i}' for i in range(1, 3)]].mean(axis=1)

mergedf['climate_risk_cnt_locations_coldwave_risk_high'] = (mergedf['medium_coldstress_4days_average'] + mergedf['high_coldstress_2days_average']) / 2
mergedf['climate_risk_cnt_locations_coldwave_risk_medium'] = (mergedf['medium_coldstress_2days_average'] + mergedf['high_coldstress_lag_1']) / 2

mergedf['climate_risk_cnt_locations_flood_risk_high'] = (mergedf['medium_precip_4days_average'] + mergedf['high_precip_2days_average']) / 2
mergedf['climate_risk_cnt_locations_flood_risk_medium'] = (mergedf['medium_precip_2days_average'] + mergedf['high_precip_lag_1']) / 2


#### 1.1 Production-Weighted Risk Scores

In [7]:
risk_categories = ['heat_stress', 'unseasonably_cold', 'excess_precip', 'drought', \
                    'coldwave', 'flood']

for risk in risk_categories:
    #low = f'climate_risk_cnt_locations_{risk}_risk_low'
    medium = f'climate_risk_cnt_locations_{risk}_risk_medium'
    high = f'climate_risk_cnt_locations_{risk}_risk_high'
    
    risk_scores = (1*mergedf[medium]+2*mergedf[high])/\
                           (mergedf['total_location_by_region'])
    ## define regional daily risk score as normalized weighted sum of number of locations
    
    production_weighted_risk_scores = (risk_scores*mergedf['percent_country_production'])/100
    ## use marketshare data to get production-weighted regional daily risk scores
    
    mergedf[f'climate_risk_{risk}_score'] = risk_scores
    mergedf[f'climate_risk_{risk}_weighted_score'] = production_weighted_risk_scores
    ## iterate for all four climate risk types; total 8 new engieered features

#### 1.2 Composite Risk Indices

In [8]:
mergedf['climate_risk_temperature_stress'] = \
mergedf[[f'climate_risk_{risk}_score' for risk in risk_categories[:2]]].max(axis=1)
## maximum of temperature-related risk scores
mergedf['climate_risk_precipitation_stress'] = \
mergedf[[f'climate_risk_{risk}_score' for risk in risk_categories[2:]]].max(axis=1)
## maximum of precipitation-related risk scores
mergedf['climate_risk_overall_stress'] = \
mergedf[[f'climate_risk_{risk}_score' for risk in risk_categories]].max(axis=1)
## maximum of all risk scores
mergedf['climate_risk_avg_stress'] = \
mergedf[[f'climate_risk_{risk}_score' for risk in risk_categories]].mean(axis=1)
## average of all risk scores
## total 4 new engineered features

#### 1.3 Risk Temporal Summaries

In [ ]:
mergedf = mergedf.sort_values(['region_name','date_on'])
window_period = [14, 20, 40, 60, 90, 120, 240]
## three periods to compute risk scores moving avg and maximum 
for window in window_period:
    for risk in risk_categories:
        mergedf[f'climate_risk_{risk}_ma_{window}d'] = \
        mergedf.groupby(['region_name'])[f'climate_risk_{risk}_score']\
               .rolling(window=window,min_periods=1).mean().reset_index(level=0,drop=True)
## compute risk score moving avg with different windows for different risk types in each region

        mergedf[f'climate_risk_{risk}_max_{window}d'] = \
        mergedf.groupby(['region_name'])[f'climate_risk_{risk}_score']\
               .rolling(window=window,min_periods=1).max().reset_index(level=0,drop=True)
## compute maximum risk scores with different windows for different risk types in each region
## total 3*4*2 = 24 new features

/var/folders/1x/j7cms8w90n71y__h4q0x74180000gn/T/ipykernel_35493/1153267851.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mergedf[f'climate_risk_{risk}_ma_{window}d'] = \
/var/folders/1x/j7cms8w90n71y__h4q0x74180000gn/T/ipykernel_35493/1153267851.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mergedf[f'climate_risk_{risk}_max_{window}d'] = \
/var/folders/1x/j7cms8w90n71y__h4q0x74180000gn/T/ipykernel_35493/1153267851.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

#### 1.4 Risk Momentum

In [10]:
features_change1d = mergedf.groupby('region_name')[[f'climate_risk_{risk}_score' for risk in risk_categories]]\
       .diff(periods=1)\
       .rename(columns=dict(zip([f'climate_risk_{risk}_score' for risk in risk_categories],\
                                [f'climate_risk_{risk}_change_1d' for risk in risk_categories])))
## Daily Change of risk scores for each risk type in each region 

features_acceleration = features_change1d.diff(periods=1)
## Acceleration of daily Change of risk scores for each risk type in each region

features_change1w = mergedf.groupby('region_name')[[f'climate_risk_{risk}_score' for risk in risk_categories]]\
       .diff(periods=7)\
       .rename(columns=dict(zip([f'climate_risk_{risk}_score' for risk in risk_categories],\
                                [f'climate_risk_{risk}_change_1d' for risk in risk_categories])))
## Weekly Change of risk scores for each risk type in each region 

mergedf = pd.concat([mergedf,\
           features_change1d,\
           features_change1w,\
           features_acceleration],axis=1)
## 12 new features in Risk Momentum category

#### 1.5 Cross-Regional features

In [11]:
feature_country = pd.concat([\
mergedf.groupby(['country_name', 'date_on'])\
[[f'climate_risk_{risk}_score' for risk in risk_categories]]\
.agg(['mean','max','std']),
## compute country-wide daily avg, max, and std risk scores
mergedf.groupby(['country_name', 'date_on'])\
[[f'climate_risk_{risk}_weighted_score' for risk in risk_categories]]\
.agg('sum')],axis=1)
## compute country-wide daily production-weighted sum risk scores
feature_country.columns = [f'climate_risk_{risk}_score_country_{metric}'\
                          for risk in risk_categories \
                          for metric in ['mean','max','std']]+\
                          [f'climate_risk_{risk}_weighted_score_country_sum'\
                          for risk in risk_categories]
## rename new features
mergedf = mergedf.merge(feature_country.reset_index(),\
              how='left',\
              on=['country_name','date_on'])
## add 4*4=16 new features

#### Non Linear Transformation
- Including Log1p, Squareroot without chaginging signs, Thresh Magnitude

In [12]:
import numpy as np

In [ ]:
climate_risk_cols = [c for c in mergedf.columns if c.startswith('climate_risk_')]
for feature_name in climate_risk_cols:
    x = mergedf.loc[:, feature_name]

    if isinstance(x, pd.DataFrame):
        x = x.iloc[:, 0]
    
    #np.log1p = ln(1+x)
    mergedf[f'{feature_name}_log1p'] = np.log1p(x.clip(lower=0))
    mergedf[f'{feature_name}_ssqrt'] = np.sign(x) * np.sqrt(np.abs(x)) # >0 <0
    mergedf[f'{feature_name}_thresh_mag'] = x.where(x>1, 0)

/var/folders/1x/j7cms8w90n71y__h4q0x74180000gn/T/ipykernel_35493/332366843.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mergedf[f'{feature_name}_thresh_mag_neg'] = x.where(x<-1, 0)
/var/folders/1x/j7cms8w90n71y__h4q0x74180000gn/T/ipykernel_35493/332366843.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mergedf[f'{feature_name}_log1p'] = np.log1p(x.clip(lower=0))
/var/folders/1x/j7cms8w90n71y__h4q0x74180000gn/T/ipykernel_35493/332366843.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the re

In [14]:
mergedf

,ID,crop_name,country_name,country_code,region_name,region_id,harvest_period,growing_season_year,date_on,climate_risk_cnt_locations_heat_stress_risk_low,...,climate_risk_drought_weighted_score_country_sum_thresh_mag,climate_risk_drought_weighted_score_country_sum_thresh_mag_neg,climate_risk_coldwave_weighted_score_country_sum_log1p,climate_risk_coldwave_weighted_score_country_sum_ssqrt,climate_risk_coldwave_weighted_score_country_sum_thresh_mag,climate_risk_coldwave_weighted_score_country_sum_thresh_mag_neg,climate_risk_flood_weighted_score_country_sum_log1p,climate_risk_flood_weighted_score_country_sum_ssqrt,climate_risk_flood_weighted_score_country_sum_thresh_mag,climate_risk_flood_weighted_score_country_sum_thresh_mag_neg
0,8308c20c-d655-4500-b41a-af7b2dd8b255,Corn: Commodity Tracked,Paraguay,PY,Alto Paraná,6a4dabfc-c598-44e0-9d99-ea9e31d110cb,Planting,2016,2016-01-01,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.708633,1.015486,1.031212,0.0
1,c6e315ff-6f9e-436d-9cb9-a2ad08b50c78,Corn: Commodity Tracked,Paraguay,PY,Alto Paraná,6a4dabfc-c598-44e0-9d99-ea9e31d110cb,Planting,2016,2016-01-02,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.441622,0.745136,0.000000,0.0
2,3e01ae6b-350a-49d5-b6d4-a6939034a280,Corn: Commodity Tracked,Paraguay,PY,Alto Paraná,6a4dabfc-c598-44e0-9d99-ea9e31d110cb,Planting,2016,2016-01-03,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.210089,0.483516,0.000000,0.0
3,719fa75b-e4b6-4d0c-ade7-fc14c8a43c49,Corn: Commodity Tracked,Paraguay,PY,Alto Paraná,6a4dabfc-c598-44e0-9d99-ea9e31d110cb,Planting,2016,2016-01-04,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.112043,0.344326,0.000000,0.0
4,06335569-7672-4ed5-b425-712d084826d0,Corn: Commodity Tracked,Paraguay,PY,Alto Paraná,6a4dabfc-c598-44e0-9d99-ea9e31d110cb,Planting,2016,2016-01-05,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320656,fae8b98a-f698-4f21-8db1-99d378b21e40,Corn: Commodity Tracked,Mexico,MX,Zacatecas,42075985-d0c8-4d2c-af41-a9efaa6826e7,Peak Harvest,2026,2025-12-11,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.113066,0.345986,0.000000,0.0
320657,3332c221-5f37-4092-b61d-9594da431479,Corn: Commodity Tracked,Mexico,MX,Zacatecas,42075985-d0c8-4d2c-af41-a9efaa6826e7,Peak Harvest,2026,2025-12-12,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.129157,0.371307,0.000000,0.0
320658,2cfa035b-7891-4acd-8081-e9e25f50bae7,Corn: Commodity Tracked,Mexico,MX,Zacatecas,42075985-d0c8-4d2c-af41-a9efaa6826e7,Peak Harvest,2026,2025-12-13,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.137704,0.384235,0.000000,0.0
320659,31328318-72ed-40ec-a31e-702e69e2d2ea,Corn: Commodity Tracked,Mexico,MX,Zacatecas,42075985-d0c8-4d2c-af41-a9efaa6826e7,Peak Harvest,2026,2025-12-14,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.136449,0.382357,0.000000,0.0


#### 1.6 Baseline features CFCS score

In [15]:
corrtable = compute_partial_correlations(mergedf)
## compute cfcs score using backtesting functionalities

In [18]:
sigcorr_report(corrtable).sort_values(by='avg_sig_corr', ascending=False).head(20)

,avg_sig_corr,max_sig_corr,sig_corr_count,sig_corr_ratio(%)
climate_variable,,,,
climate_risk_coldwave_max_40d_thresh_mag,0.700,0.782,13,0.814
climate_risk_coldwave_max_60d_thresh_mag,0.671,0.781,24,1.486
climate_risk_drought_ma_120d_thresh_mag,0.626,0.781,10,0.964
climate_risk_coldwave_ma_14d,0.622,0.632,2,0.110
climate_risk_coldwave_max_90d_thresh_mag,0.621,0.825,24,1.384
climate_risk_coldwave_ma_14d_log1p,0.619,0.629,2,0.110
climate_risk_coldwave_max_14d_thresh_mag,0.619,0.757,6,0.447
climate_risk_excess_precip_max_240d_log1p,0.611,0.800,58,2.585
climate_risk_excess_precip_max_240d_ssqrt,0.610,0.798,58,2.585


In [17]:
cfcs(corrtable)

0.44% of all correlations are significant
Average significant correlation is 0.558
highest absolute correlation found is 0.825
final CFCS score is 52.74


{'cfcs_score': np.float64(52.73791054970592),
 'avg_sig_score': np.float64(55.77694369256475),
 'max_corr_score': np.float64(82.54),
 'sig_count_score': 0.4371935171177515}

In [ ]:
features_sig = [
'climate_risk_coldwave_max_40d_thresh_mag',
'climate_risk_coldwave_max_60d_thresh_mag',
'climate_risk_coldwave_max_90d_thresh_mag',
'climate_risk_excess_precip_max_240d_log1p',
'climate_risk_excess_precip_max_240d_ssqrt',
'climate_risk_excess_precip_max_240d',
]	

submissiondf = mergedf.dropna()
submissiondf = submissiondf[submissiondf.columns[pd.Series(submissiondf.columns).apply(\
lambda x:( not x.startswith('climate_risk')) or (x in features_sig)
)]]

In [ ]:
corrtable = compute_partial_correlations(submissiondf)

In [ ]:
cfcs(corrtable)

1.98% of all correlations are significant
Average significant correlation is 0.623
highest absolute correlation found is 0.825
final CFCS score is 56.31


{'cfcs_score': np.float64(56.312200861130485),
 'avg_sig_score': np.float64(62.30923809523811),
 'max_corr_score': np.float64(82.54),
 'sig_count_score': 1.9779090675571538}